In [1]:
import pandas as pd
import torch

In [2]:
train = pd.read_csv('/home/jiajunb/neural-dimension-reduction/data/processed/sample/train.csv', header=None)

In [3]:
train

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,95.195544,12.221859,47.141783,75.731650,68.580790,83.656304,16.142661,10.761436,24.584586,64.646126,...,55.934230,81.640514,60.744191,97.305761,7.392480,13.650104,22.542291,29.720707,37.517233,4.450282
1,99.166320,15.227976,44.293374,78.179281,64.172340,85.974963,12.933176,12.588127,27.508371,65.913688,...,55.372351,82.652559,59.682662,93.990164,7.141942,15.400499,17.470659,26.153575,38.005573,3.043574
2,43.596314,51.122708,22.307158,47.822319,91.865922,45.266854,93.859339,75.470586,40.011441,48.181460,...,62.268754,65.520484,63.083167,57.314553,36.625940,19.176103,58.446734,6.461213,21.434521,28.600447
3,83.726750,44.311964,38.018275,7.328273,88.471305,39.343504,9.586104,26.378220,13.760890,84.802381,...,39.916273,33.357095,26.539032,40.089722,66.134136,51.373676,18.746099,48.416866,64.521661,31.214616
4,7.538269,96.397863,27.843163,89.924001,78.159261,78.381714,14.137873,67.815051,89.252998,53.251439,...,86.379488,93.653028,87.877870,-0.110282,85.945464,30.948948,22.571470,83.076293,7.505366,33.383922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,95.988460,96.597039,6.417373,11.864794,29.096424,51.156862,70.729162,0.053391,47.769190,8.089639,...,33.746159,84.972306,78.887586,22.991115,86.330459,67.422775,12.524036,0.086390,30.022850,45.891306
1996,98.950657,96.770066,4.619809,11.878924,30.487113,49.628742,69.548092,0.311771,45.514677,8.905408,...,32.391831,82.134732,79.151044,23.002295,84.818998,70.569570,12.428116,-0.077022,28.128708,45.544241
1997,44.181214,7.364665,47.728003,46.507556,92.448435,84.817303,85.524844,20.197726,56.884943,21.684099,...,43.780489,47.282330,42.974455,54.554699,63.130056,23.894714,12.693856,25.860484,81.959900,19.465354
1998,87.065177,41.652331,36.694405,6.778014,90.055636,36.665053,9.999484,24.324081,15.838263,84.286281,...,37.389795,33.416162,29.278542,40.517507,63.278387,48.665805,15.918664,49.526287,66.486111,35.752724


In [82]:
import torch
from torch.nn import functional as F
# from tqdm.auto import tqdm


def pairwise_distance(x):
    """
    calculate the nearest neighbors of x, return the
    :param x: for matrix to calculate nearest neighbor
    :return:
        dist: pairwise distances
    """
    batch_size = 2000
    if x.shape[0] * x.shape[1] < batch_size * 200:  # direct computes the whole matrix
        dist = torch.cdist(x1=x, x2=x, p=2)  # (n, n)
    else:  # calculate the nearest neighbors in batches
        num_iter = x.shape[0] // batch_size + 1
        dist = list()
        for i in torch.arange(num_iter):
            batch_x = x[i * batch_size: (i + 1) * batch_size, :]
            dist.append(torch.cdist(x1=batch_x, x2=x, p=2))  # (n, n)
        dist = torch.cat(dist, dim=0)
    return dist


def softmin_probability(xdist_batch, sigma):
    n = xdist_batch.shape[0]
    idx = torch.arange(n)
    d = -xdist_batch.square() / 2 / (sigma ** 2)
    d = d - torch.mean(d)
    p = d.exp()
    
    p[idx, idx] = 0
    p = p / p.sum(dim=1, keepdim=True)
    p[idx, idx] = 0
    return p


def calculate_perplexity(prob):
    n = prob.shape[0]
    idx = torch.arange(n)
    entropy = torch.log2(prob) * prob
    entropy[idx, idx] = 0
    hp = -torch.sum(entropy, dim=1)
    print('hp', hp)
    print(torch.log2(prob) * prob)
    return 2 ** hp


def get_input_probability(x_dist_batch, ppl):
    sigma = 1.
    data_ppl = calculate_perplexity(softmin_probability(x_dist_batch, sigma))
    while (data_ppl - ppl).norm(p=2) < 1e-3:
        sigma *= 2
    return sigma


def get_output_probability(y, alpha):
    pass

In [83]:
dist = pairwise_distance(torch.from_numpy(train.to_numpy()))

In [84]:
(dist ** 2).max()

tensor(411710.4468, dtype=torch.float64)

In [85]:
(dist ** 2).min()

tensor(0., dtype=torch.float64)

In [86]:
torch.exp(0-(dist ** 2) / 2 / (5e2 ** 2)).mean()

tensor(0.5675, dtype=torch.float64)

In [91]:
x = softmin_probability(dist, 30)
print(x)
ppl = calculate_perplexity(x)
print(ppl)

tensor([[0.0000e+00, 5.5308e-03, 1.1015e-81,  ..., 6.3476e-80, 1.2650e-74,
         4.6448e-85],
        [5.6170e-03, 0.0000e+00, 1.4080e-80,  ..., 1.6410e-79, 4.9491e-74,
         2.1821e-83],
        [1.0393e-81, 1.3081e-80, 0.0000e+00,  ..., 2.8777e-76, 4.4245e-87,
         1.3808e-87],
        ...,
        [7.5351e-80, 1.9181e-79, 3.6205e-76,  ..., 0.0000e+00, 2.1126e-80,
         2.0255e-88],
        [9.5166e-75, 3.6661e-74, 3.5279e-87,  ..., 1.3389e-80, 0.0000e+00,
         3.4107e-82],
        [4.5517e-85, 2.1056e-83, 1.4342e-87,  ..., 1.6721e-88, 4.4427e-82,
         0.0000e+00]], dtype=torch.float64)
hp tensor([7.5798, 7.5812, 7.7171,  ..., 7.6934, 7.7785, 7.4232],
       dtype=torch.float64)
tensor([[        nan, -4.1472e-02, -2.9624e-79,  ..., -1.6700e-77,
         -3.1053e-72, -1.3012e-82],
        [-4.1993e-02,         nan, -3.7349e-78,  ..., -4.2948e-77,
         -1.2052e-71, -5.9920e-81],
        [-2.7960e-79, -3.4713e-78,         nan,  ..., -7.2213e-74,
         -1.2692

In [ ]:
x_dist_batch.square() / 2 / (30 ** 2)